<div style="text-align: center;">
  <h1>TA136 - Taller de Procesamiento de Señales</h1>
  <h2>Trabajo Práctico 10: Multinomial Naive Bayes</h2>
</div>

---
---

<div style="text-align: center;">
  <h3> Introducción
</div>

&ensp; El presente trabajo práctico tiene como objetivo implementar y utilizar un clasificador *Multinomial Naive Bayes* para predecir la veracidad y orientación política de artículos periodísticos. Para ello, se utiliza la base de datos *BuzzFeed-Webis Fake News Corpus 2016*, que contiene noticias vinculadas a las elecciones estadounidenses de ese año.

&ensp; El desarrollo se estructura en dos etapas principales: en primer lugar, se preprocesa el texto utilizando $\texttt{CountVectorizer}$, para así obtener una matriz de ocurrencias de palabras dentro del vocabulario dado por los artículos. A fin de realizar esto, se transformaron los textos a minúsculas, se eliminaron *stopwords* y se filtraron términos muy o poco frecuentes. Luego,  se implementa una clase que representa el clasificador *MNB*, la cual contempla la etapa de entrenamiento y predicciones *soft* y *hard*.

&ensp; En la segunda parte, se realiza una evaluación del desempeño del modelo. Para ello, se emplean las métricas *accuracy* y Macro-F1 sobre un conjunto de testeo separado previamente. Se realiza este análisis tanto para la clasificación de la veracidad del artículo como para la orientación política del medio, observando diferencias del modelo ante distintas clases.

---
---

<div style="text-align: center;">
  <h3> Desarrollo
</div>

**La base de datos *BuzzFeed-Webis Fake News Corpus 2016* posee diferentes artículos periodísticos de una semana cercana a las elecciones estadounidenses de ese año. Se desea entrenar un algoritmo *Multinomial Naive Bayes* capaz de clasificar los artículos en: “*mayormente falso*”, “*mayormente verdadero*”, “*mezcla de verdadero y falso*” y “*sin contenido factual*”.**

---

#### (A). *Exploración de Datos:*

- **Descargar la base de datos en $\texttt{https://zenodo.org/record/1239675/files/articles.zip?download=1.} $**

&ensp; Con el objetivo de acondicionar los datos a utilizar, se usa la librería `os`, que permite ejecutar comandos de $\texttt{Linux}$. Así, se descarga el $\texttt{.zip}$ de datos.

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [16]:
os.system("wget https://zenodo.org/record/1239675/files/articles.zip")
os.system("unzip articles.zip")

256

- **Construir la base de datos.**

&ensp; Luego, a partir del código proporcionado por la cátedra, se construye la base de datos y se la imprime por pantalla, para verificar que la forma sea la esperada.

In [17]:
import xml.etree.ElementTree as ET

data = {"mainText": [], "orientation": [], "veracity": []}
for filename in os.listdir("articles/"):
  root = ET.parse(f"articles/{filename}").getroot()
  for elem in root:
    if elem.tag in data.keys():
      data[elem.tag].append(elem.text)

data = pd.DataFrame(data)
data = data[data.notna().all(axis="columns")]
display(data)


,mainText,orientation,veracity
0,Senator Mark Kirk (R-IL) has been caught red-h...,left,mostly true
1,The 2016 election is shaping up to be the weir...,left,mixture of true and false
2,"CNN recently ran an article titled, “Donald Tr...",right,mixture of true and false
3,Clowns are on a rampage along the east coast. ...,right,mostly true
4,Donald Trump's presidential transition operati...,mainstream,mostly true
...,...,...,...
1622,Charlotte riots have gone mad. The idiots ther...,right,mostly false
1623,Atlanta (CNN)Long before New Jersey Gov. Chris...,mainstream,mostly true
1624,Hillary Clinton will almost certainly win Mond...,mainstream,mostly true
1625,The top official in the country’s largest poli...,mainstream,mostly true


- **Utilice el comando $\texttt{train\_test\_split}$ (`sklearn`) para definir dos conjuntos de datos. El conjunto de entrenamiento debe contener el $80 \%$ de las muestras, el resto serán de testeo.**

&ensp; En base a la función recomendada por la cátedra, se separan los conjuntos de datos en dos: uno para el entrenamiento y otro para el testeo del algoritmo a realizar. Se verifica que las dimensiones de los conjuntos sean coherentes con lo solicitado.

In [18]:
from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(data, train_size=0.8)

print("train shape:", data_train.shape)
print("test shape:", data_test.shape)

train shape: (1283, 3)
test shape: (321, 3)


&ensp; Una vez hecho esto, se definen las $X$'s e $y$'s de entrenamiento y testeo, según su correspondiente conjunto. De esta manera, se obtienen tanto las clases correspondientes a la veracidad de un artículo como a la orientación política del mismo.

In [19]:
X_train = data_train["mainText"]
X_test = data_test["mainText"]

y_ver_train = data_train["veracity"]
y_ver_test = data_test["veracity"]

y_ori_train = data_train["orientation"]
y_ori_test = data_test["orientation"]

- **Utilizando $\texttt{CountVectorizer}$ (`sklearn`) pre-procesar los datos del texto principal de los artículos.**

&ensp; En función de lo sugerido por la cátedra, se pre-procesan los datos para así obtener una matriz que contabiliza cada una de las palabras del vocabulario compuesto por los artículas. Para ello, se eliminan las *stop words*, las palabras que aparecen más del $60 \%$ de los documentos y las que se encuentran en menos de $3$; esto se debe principalmente a que estas no ayudarán a la hora de realizar la clasificación.

&ensp; Cabe destacar que los datos de testeo se transforman de acuerdo al entrenamiento del $\texttt{CountVectorizer}$ con el conjunto de entrenamiento. Además, esta función devuelve una matriz dispersa, que es computacionalmente más eficiente, pero es difícil de manejar al realizar operaciones. Por lo tanto, se cambia a un tipo de dato más común como el *array*.

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(lowercase=True, stop_words='english', max_df=0.6, min_df=3)
X_train_vec = vectorizer.fit_transform(X_train).toarray()
X_test_vec = vectorizer.transform(X_test).toarray()

In [21]:
print("X_train_vec shape:", X_train_vec.shape)
print("X_test_vec shape:", X_test_vec.shape)

X_train_vec shape: (1283, 11375)
X_test_vec shape: (321, 11375)


---

#### (B). *Entrenamiento:*

**Implementar un *MNB* de $\alpha = (1, ~ 1, ~ \dots ~ , ~ 1)$ que prediga la veracidad de un artículo a partir de su texto principal (pre-procesado). El código debe estar estructurado de la siguiente manera:**
```python
class MNB:
    # Inicializar atributos y declarar hiperparámetros
    def __init__(self, ...

    # Etapa de entrenamiento
    def fit(self, X, y):
    
    # Etapa de testeo soft
    def predict_proba(self, X):
    
    # Etapa de testeo hard (no repetir código)
    def predict(self, X):
```

&ensp; Previo a comentar lo realizado a nivel código, se realiza un paneo teórico de lo visto en las clases del curso. En este caso, el tema a tratar es el de *Multinomial Navie Bayes*. Para ello, se menciona que se lo va a utilizar para clasificar un texto según su tópico y se definen las siguientes variables:

- $k$: cantidad de tópicos.
- $V$: cantidad de palabras del vocabulario.
- $d$: cantidad de palabras del texto.
- $Y_i$: tópico al que pertenece el texto.
- $\theta_i^{(k)}$: probabilidad de que la palabra sea la $i$-ésima en la clase $k$.
- $N_m$: cantidad de veces que aparece la palabra $m$-ésima.

&ensp; Así, se definen las probabilidades de las clases ($c_y$) y las $N_m$ indexadas según la clase $k$-ésima a la que se pertenece ($\tilde{N}_m^{(k)}$):

\begin{align*}
    \begin{cases}
        c_y = \frac{\# \{ y_i = k \}}{n} \\
        \tilde{N}_m^{(k)} = \sum_{i = 1}^{n} N_{i, ~ m} \cdot 𝟙\{ y_i = k \}
    \end{cases}
\end{align*}

&ensp; De esta manera, se parte de las dos suposiciones mostradas a continuación.

\begin{align*}
    T=[T_1, ~ \dots ~, ~ T_V] &\sim \text{Dir}(\alpha_1, ~ \dots ~, ~\alpha_V) \\
    \left( \tilde{N}_1^{(k)}, ~ \dots ~, ~ \tilde{N}_V^{(k)} \right) ~ \bigg| ~  T &\sim \mathcal{M} \left(\tilde{d}^{(k)}, ~[\theta_1^k, ~ \dots ~, ~\theta_V^k] \right)
\end{align*}

donde el vector aleatorio $T$ representa las realizaciones $\theta_i^{(k)}$ y puede reescribirse como una distribución beta multivariada, tal que sus marginales son de la forma:

$$T_m \sim \beta \left(\alpha_m, \sum_{\eta \neq m} \alpha_\eta \right).$$

&ensp; Entonces, desarrollando la distribución a posteriori de $T$ dados los $\tilde{N}_m^{(k)}$, se obtiene que dicha distribución se corresponde con la siguiente Dirichlet:

\begin{align*}
    p\left(T ~ \bigg| ~ \left[ \tilde{N}_1^{(k)}, ~ \dots ~, ~ \tilde{N}_V^{(k)} \right] \right) \propto p\left( \left[\tilde{N}_1^{(k)}, ~ \dots ~, ~ \tilde{N}_V^{(k)} \right] ~ \bigg| ~  T  \right) \cdot p(T) \\
    \Longrightarrow T ~ \bigg| ~ \left[ \tilde{N}_1^{(k)}, ~ \dots ~, ~ \tilde{N}_V^{(k)} \right] \sim \text{Dir} \left( \left[ \tilde{N}_1^{(k)} + \alpha_1, ~ \dots ~, ~ \tilde{N}_V^{(k)} + \alpha_V \right] \right)
\end{align*}

dicha distribución, al igual que anteriormente, puede ser representada marginalmente como una beta. Por lo tanto, al analizar la media, se obtiene el estimador bayesiano de cada una de las realizaciones.

\begin{align*}
    T_m ~ \bigg| ~ \left[ \tilde{N}_1^{(k)}, ~ \dots ~, ~ \tilde{N}_V^{(k)} \right] \sim \beta \left(  \tilde{N}_m^{(k)} + \alpha_m, ~ \sum_{\eta \neq m} \tilde{N}_\eta^{(k)} + \alpha_\eta \right) \\
    \Longrightarrow \hat{\theta}_m^{(k)} = E \left[ T_m ~ \bigg| ~ \left[ \tilde{N}_1^{(k)}, ~ \dots ~, ~ \tilde{N}_V^{(k)} \right] \right] = \frac{\tilde{N}_m^{(k)} + \alpha_m}{ \sum_{\eta = 1}^V \tilde{N}_\eta^{(k)} + \alpha_\eta}
\end{align*}

&ensp; Habiendo obtenido los parámetros de entrenamiento, se procede a realizar la inferencia de los datos según la expresión:

$$ \max_y ~ p(y ~| ~ x) = \max_y ~ \log(c_y) + \sum_{m = 1}^V N_m \cdot \log\left( \theta_m^{(k)} \right)$$

&ensp; Una vez presentados estos conceptos y a fin de desarrollar el algoritmo, se siguieron los lineamientos dados por la cátedra y se definió la clase llamada $\texttt{MNB}$. Esta implementa los métodos que se detallan a continuación:

- `__init__:` Inicializa la clase y declara los atributos necesarios para almacenar los parámetros del modelo: las clases, la cantidad de clases, la probabilidad de cada clase, los $\theta$ y los $N_m$ indexados por cada clase.

- `fit:` A partir de las expresiones vistas anteriormente para las $c_y$, las $\tilde{N}_m^{(k)}$ y los $\hat{\theta}_m^{(k)}$; se procede a realizar el entrenamiento del modelo.

- `predict_proba:` En base a la expresión presentada con antelación para la inferencia, se predicen las probabilidades de cada texto a pertenecer a una clase en particular. Se utiliza la función $\texttt{softmax}$ para obtener las probabilidades correspondientes a partir de sus logaritmos.

- `predict:` En función del método $\texttt{predict_proba}$, se realiza una predicción *hard* donde se devuelve la clase cuya probabilidad sea la más alta.

In [22]:
from scipy.special import softmax

In [23]:
class MNB:
    def __init__(self):
        self.classes = None
        self.n_classes = None
        self.class_proba = None
        self.theta = None

    def fit(self, X, y):
        self.classes, counts = np.unique(y, return_counts=True)
        self.n_classes = len(self.classes)
        self.class_proba = counts / y.shape[0]
        Nm = np.array([np.sum(X[y == self.classes[i]], axis=0) for i in range(len(self.classes))])
        self.theta = (Nm + 1) / (np.sum(Nm + 1, axis=1, keepdims=True))

    def predict_proba(self, X):
        log_probs = X @ np.log(self.theta).T + np.log(self.class_proba)
        return softmax(log_probs, axis=1)

    def predict(self, X):
        return self.classes[np.argmax(self.predict_proba(X), axis=1)]

    def accuracy(self, X, y):
        return np.mean(self.predict(X) == y)

    def macro_f1(self, X, y):
        y_pred = self.predict(X)
        f1 = np.zeros(self.n_classes)
        for i in range(self.n_classes):
            tp = np.sum((y_pred == self.classes[i]) & (y == self.classes[i]))
            fp = np.sum((y_pred == self.classes[i]) & (y != self.classes[i]))
            fn = np.sum((y_pred != self.classes[i]) & (y == self.classes[i]))

            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1[i] = 2 * precision * recall / (precision + recall) if precision + recall != 0 else 0

        return np.mean(f1)

---

&ensp; Luego, se procede a inicializar la clase y entrenarla con el objetivo de que prediga la veracidad de un archivo.

In [24]:
MNB_ver = MNB()
MNB_ver.fit(X_train_vec, y_ver_train)

#### (C). *Inferencia:*

**Implementar un método a la clase anterior que calcule el accuracy y la Macro-F1. Evaluar dichas métricas en el conjunto de testeo. ¿Por qué dan tan diferentes?**

&ensp; Se le agregan dos métodos más a la clase $\texttt{MNB}$, uno correspondiente a la *accuracy* y otro a la Macro-F1. A continuación, se detallan sus características:

- `accuracy:` Evalúa el desempeño del modelo comparando las etiquetas predichas con las verdaderas. Se calcula de la siguiente manera:
$$\text{accuracy} = \frac{1}{n} \sum_{i=1}^n 𝟙\{ \hat{y}_i = y_i \}$$

- `macro_f1:` Mide el desempeño del modelo promediando el F1-score calculado para cada clase $k$. Así, se define:
\begin{align*}
    \begin{cases}
    \text{precision}^{(k)} = \frac{TP}{TP + FP} \\
    \text{recall}^{(k)} = \frac{TP}{TP + FN}
    \end{cases}
    \Longrightarrow \text{F1}^{(k)} = \frac{2 \cdot \text{precision}^{(k)} \cdot \text{recall}^{(k)}}{\text{precision}^{(k)} + \text{recall}^{(k)}}
\end{align*}
donde la Macro-F1 se obtiene como:
$$\text{Macro-F1} = \frac{1}{n} \cdot \sum_{k = 1}^n \text{F1}^{(k)} $$

&ensp; De esta manera, se calculan ambas métricas según el modelo entrenado anteriormente para predecir la veracidad de un texto.


In [25]:
print(f"Accuracy veracity: {MNB_ver.accuracy(X_test_vec, y_ver_test):.3f}")
print(f"Macro-F1 veracity: {MNB_ver.macro_f1(X_test_vec, y_ver_test):.3f}")

Accuracy veracity: 0.688
Macro-F1 veracity: 0.322


&ensp; En este caso, la diferencia entre las métricas se debe principalmente a un fuerte desbalance en las clases, siendo *mostly true* la categoría mayoritaria. Como consecuencia, el modelo tiende a predecir esta clase con frecuencia, lo que resulta en una *accuracy* elevada. Sin embargo, la macro-F1 evalúa el desempeño promedio en cada clase dándole igual peso a todas, por lo que penaliza fuertemente los errores en clases minoritarias. Esto explica su valor considerablemente más bajo.

&ensp; Así, se pueden observar las clases y sus probabilidades, verificando que esto en verdad ocurre.

In [26]:
display(pd.DataFrame({'Class': MNB_ver.classes, 'Probability': MNB_ver.class_proba}))

,Class,Probability
0,mixture of true and false,0.130164
1,mostly false,0.049883
2,mostly true,0.784100
3,no factual content,0.035853


---

#### (D). *Orientación:*

**Repetir el ejercicio pero para clasificar la orientación política del portal donde fue publicada la noticia (izquierda, derecha o *mainstream*) a partir del texto principal preprocesado. ¿Siguen siendo válidas las conclusiones extraídas anteriormente? Justificar.**

&ensp; A fin de realizar este inciso, se entrena el mismo modelo con las clases que respectan a la orientación política del portal donde se publicó la noticia. Luego, se utilizan los métodos descriptos anteriormente y se calcula su *accuracy* y su Macro-F1.

In [27]:
MNB_ori = MNB()
MNB_ori.fit(X_train_vec, y_ori_train)

print(f"Accuracy orientation: {MNB_ori.accuracy(X_test_vec, y_ori_test):.3f}")
print(f"Macro-F1 orientation: {MNB_ori.macro_f1(X_test_vec, y_ori_test):.3f}")

Accuracy orientation: 0.854
Macro-F1 orientation: 0.811


&ensp; Aquí, se puede visualizar posteriormente que las probabilidades de las clases son comparables. Entonces, el modelo no predice a una única clase, sino que aprende a distinguirlas correctamente. Como resultado, tanto la *accuracy* como la Macro-F1 reflejan un rendimiento coherente y similar, ya que el desempeño es similar en todas las clases. Por ende, las conclusiones extraídas anteriormente no son válidas en este caso.

In [28]:
display(pd.DataFrame({'Class': MNB_ori.classes, 'Probability': MNB_ori.class_proba}))

,Class,Probability
0,left,0.159782
1,mainstream,0.505066
2,right,0.335152


---
---

<div style="text-align: center;">
  <h3> Conclusiones
</div>

&ensp; La implementación del clasificador *Multinomial Naive Bayes* permitió predecir tanto la veracidad del contenido como la orientación política de artículos de prensa, utilizando lo visto en las clases teóricas del curso.

&ensp; Los resultados obtenidos mostraron que, si bien la *accuracy* fue razonablemente alta en ambos casos, la Macro-F1 reveló una baja capacidad del modelo para tratar con clases desbalanceadas como la de la veracidad del artículo.

&ensp; En definitiva, el trabajo permitió observar las limitaciones y ventajas del *MNB*. Entre sus ventajas se destacan su simplicidad, eficiencia computacional y facilidad de implementación. Sin embargo, también se observó que tiende a favorecer la clase con mayor probabilidad, lo cual puede traer problemas cuando las clases no están equilibradas, como se mencionó anteriormente.